In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pygad
import numpy
import pygad.nn
import pygad.gann
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import tensorflow as tf
import random
import mplcyberpunk

In [48]:
df = pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [334]:
y = df.output
X = df.drop(columns="output")

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify = y,
                                                    test_size = .2,
                                                    random_state = 42)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((242, 13), (242,), (61, 13), (61,))

In [238]:
mean_population_fitness = []
def fitness_func(ga_instance, solution, sol_idx):
    global GANN_instance, data_inputs, data_outputs
    if sol_idx is not None:
        predictions = pygad.nn.predict(last_layer=GANN_instance.population_networks[sol_idx],data_inputs=data_inputs)
        correct_predictions = numpy.where(predictions == data_outputs)[0].size
        solution_fitness = (correct_predictions/data_outputs.size)*100
        mean_population_fitness.append(solution_fitness)
    else:
        solution_fitness=0
    return solution_fitness


In [239]:
last_fitness = 0

def callback_generation(ga_instance):
    global GANN_instance, last_fitness

    population_matrices = pygad.gann.population_as_matrices(population_networks=GANN_instance.population_networks,
                                                            population_vectors=ga_instance.population)

    GANN_instance.update_population_trained_weights(population_trained_weights=population_matrices)
    if ga_instance.generations_completed % verbose == 0:
        print("Generation = {generation}".format(generation=ga_instance.generations_completed))
        print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
        print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))

    last_fitness = ga_instance.best_solution()[1].copy();

In [240]:
data_inputs = X_train
data_outputs = y_train.values

num_inputs = data_inputs.shape[1]
num_classes = 2

In [341]:
GANN_instance = pygad.gann.GANN(num_solutions=num_solutions,
                                num_neurons_input=num_inputs,
                                num_neurons_hidden_layers=[8,16,32],
                                num_neurons_output=num_classes,
                                hidden_activations=["relu", "relu","relu"],
                                output_activation="sigmoid"
                               )
population_vectors = pygad.gann.population_as_vectors(population_networks=GANN_instance.population_networks)

initial_population = population_vectors.copy()

avg_acc_score = 0

for x in range(num_solutions - 1):
    y_pred_ts = pygad.nn.predict(last_layer=GANN_instance.population_networks[x], data_inputs=X_test)
    avg_acc_score += accuracy_score(y_test, y_pred_ts)
print('Average Accuracy Test Score: {0:0.2f}'.format(avg_acc_score / num_solutions)) 

Average Accuracy Test Score: 0.50


In [320]:
num_solutions = 50
num_generations = 600

parent_selection_type = "rws"
keep_parents = -1

crossover_type = "two_points"

mutation_type = "adaptive"

num_parents_mating = 4

crossover_probability = 0.4

In [332]:
best_all_gen = []
best_fitness_individual = 0
best_test_accuracy = 0
verbose = 100
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       mutation_percent_genes=(25,10),
                       mutation_probability=(0.15,0.05),
                       mutation_num_genes=(4,2),
                       num_genes=len(X_train),
                       parent_selection_type=parent_selection_type,
                       crossover_probability=crossover_probability,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       keep_parents=keep_parents,
                       on_generation=callback_generation)

ga_instance.run()

ga_instance.plot_fitness()

solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
y_pred_ts = pygad.nn.predict(last_layer=GANN_instance.population_networks[solution_idx],
                               data_inputs=X_test)
test_acc = accuracy_score(y_test, y_pred_ts)
if test_acc > best_test_accuracy:
    best_test_accuracy = test_acc
        
if solution_fitness > best_fitness_individual:
    best_fitness_individual = solution_fitness
best_all_gen.append(solution_fitness)

if ga_instance.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

predictions = pygad.nn.predict(last_layer=GANN_instance.population_networks[solution_idx],
                                data_inputs=data_inputs)

D:\Anaconda\lib\site-packages\pygad\nn\nn.py:156: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + numpy.exp(-1 * sop))


KeyboardInterrupt: 

In [322]:
import statistics

num_wrong = numpy.where(predictions != data_outputs)[0]
num_correct = data_outputs.size - num_wrong.size
accuracy = 100 * (num_correct/data_outputs.size)
print("Number of correct classifications : {num_correct}.".format(num_correct=num_correct))
print("Number of wrong classifications : {num_wrong}.".format(num_wrong=num_wrong.size))
print("Classification accuracy : {accuracy}.".format(accuracy=accuracy))
print("Mean population fitness: {mean_fitness}".format(mean_fitness=statistics.mean(mean_population_fitness)))

Number of correct classifications : 208.
Number of wrong classifications : 34.
Classification accuracy : 85.9504132231405.
Mean population fitness: 64.04278901797927


In [323]:
y_pred_tr = pygad.nn.predict(last_layer=GANN_instance.population_networks[solution_idx],
                               data_inputs=X_train)
y_pred_ts = pygad.nn.predict(last_layer=GANN_instance.population_networks[solution_idx],
                               data_inputs=X_test)

print('Train Accuracy score: {0:0.4f}'.format(accuracy_score(y_train, y_pred_tr)))
print('Test Accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred_ts)))

Train Accuracy score: 0.8595
Test Accuracy score: 0.6557


In [324]:
best_all_tests.append(best_all_gen)
fitness_all_tests.append(best_fitness_individual)
average_y_accuracy.append(best_test_accuracy)

print(best_all_tests, fitness_all_tests)

[[86.77685950413223, 87.60330578512396, 87.60330578512396, 85.9504132231405, 88.42975206611571, 85.9504132231405, 87.60330578512396, 87.60330578512396, 87.60330578512396, 86.77685950413223], [88.84297520661157, 85.53719008264463, 88.01652892561982, 87.60330578512396, 82.64462809917356, 85.12396694214877, 88.01652892561982, 86.36363636363636, 87.19008264462809, 86.77685950413223], [86.36363636363636, 84.71074380165288, 87.60330578512396, 87.60330578512396, 85.53719008264463, 87.19008264462809, 85.9504132231405, 86.77685950413223, 86.77685950413223, 85.9504132231405]] [88.42975206611571, 88.84297520661157, 87.60330578512396]
